In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions
from pyspark.sql.types import StructType
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql.types import FloatType, IntegerType, DateType
from pyspark.sql.functions import udf, isnan, when, count, col

In [2]:
import os 
#os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /usr/share/java/mariadb-java-client-2.5.3.jar pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /Users/majo/Documents/david/mariadb-java-client-2.5.3.jar pyspark-shell'

In [3]:
spark_context = SparkContext()
sql_context = SQLContext(spark_context)
spark = sql_context.sparkSession

21/10/24 22:17:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
/Users/majo/opt/anaconda3/envs/Modelo_ETL/lib/python3.8/site-packages/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [4]:
PATH = "/Users/majo/Desktop/Maestría_MIAD/Semestre_1/Modelado_de_Datos_y_ETL/Semana_2/Tarea_Data_Profiling/"

In [5]:
aeropuertos = spark.read.load(PATH+"aeropuertos.csv",format="csv", sep=",", inferSchema="true", header="true")
aeropuertos.show()

+---+-----+----+--------------------+--------------------+------------------+-------------+-------+--------+--------------------+--------------------+--------------+-----------+-------+---------+----------+------------------+--------------+-----+----------+--------------------+----------------+-------------+
|_c0|sigla|iata|              nombre|           municipio|      departamento|    categoria|latitud|longitud|         propietario|          explotador|longitud_pista|ancho_pista|   pbmo|elevacion|resolucion|fecha_construccion|fecha_vigencia|clase|      tipo|numero_vuelos_origen|gcd_departamento|gcd_municipio|
+---+-----+----+--------------------+--------------------+------------------+-------------+-------+--------+--------------------+--------------------+--------------+-----------+-------+---------+----------+------------------+--------------+-----+----------+--------------------+----------------+-------------+
|274|  BOG| BOG|           EL DORADO|        Bogotá, D.C.|      Bogotá

21/10/24 22:17:19 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , sigla, iata, nombre, municipio, departamento, categoria, latitud, longitud, propietario, explotador, longitud_pista, ancho_pista, pbmo, elevacion, resolucion, fecha_construccion, fecha_vigencia, clase, tipo, numero_vuelos_origen, gcd_departamento, gcd_municipio
 Schema: _c0, sigla, iata, nombre, municipio, departamento, categoria, latitud, longitud, propietario, explotador, longitud_pista, ancho_pista, pbmo, elevacion, resolucion, fecha_construccion, fecha_vigencia, clase, tipo, numero_vuelos_origen, gcd_departamento, gcd_municipio
Expected: _c0 but found: 
CSV file: file:///Users/majo/Desktop/Maestría_MIAD/Semestre_1/Modelado_de_Datos_y_ETL/Semana_2/Tarea_Data_Profiling/aeropuertos.csv


In [6]:
print(aeropuertos.schema)

StructType(List(StructField(_c0,IntegerType,true),StructField(sigla,StringType,true),StructField(iata,StringType,true),StructField(nombre,StringType,true),StructField(municipio,StringType,true),StructField(departamento,StringType,true),StructField(categoria,StringType,true),StructField(latitud,DoubleType,true),StructField(longitud,DoubleType,true),StructField(propietario,StringType,true),StructField(explotador,StringType,true),StructField(longitud_pista,DoubleType,true),StructField(ancho_pista,DoubleType,true),StructField(pbmo,StringType,true),StructField(elevacion,DoubleType,true),StructField(resolucion,StringType,true),StructField(fecha_construccion,StringType,true),StructField(fecha_vigencia,StringType,true),StructField(clase,StringType,true),StructField(tipo,StringType,true),StructField(numero_vuelos_origen,StringType,true),StructField(gcd_departamento,IntegerType,true),StructField(gcd_municipio,IntegerType,true)))


In [7]:
row_number = aeropuertos.count()
print(row_number)

292


In [8]:
aeropuertos.select([count(when(isnan(c), c)).alias(c) for c in aeropuertos.columns]).show()

21/10/24 22:17:25 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , sigla, iata, nombre, municipio, departamento, categoria, latitud, longitud, propietario, explotador, longitud_pista, ancho_pista, pbmo, elevacion, resolucion, fecha_construccion, fecha_vigencia, clase, tipo, numero_vuelos_origen, gcd_departamento, gcd_municipio
 Schema: _c0, sigla, iata, nombre, municipio, departamento, categoria, latitud, longitud, propietario, explotador, longitud_pista, ancho_pista, pbmo, elevacion, resolucion, fecha_construccion, fecha_vigencia, clase, tipo, numero_vuelos_origen, gcd_departamento, gcd_municipio
Expected: _c0 but found: 
CSV file: file:///Users/majo/Desktop/Maestría_MIAD/Semestre_1/Modelado_de_Datos_y_ETL/Semana_2/Tarea_Data_Profiling/aeropuertos.csv


+---+-----+----+------+---------+------------+---------+-------+--------+-----------+----------+--------------+-----------+----+---------+----------+------------------+--------------+-----+----+--------------------+----------------+-------------+
|_c0|sigla|iata|nombre|municipio|departamento|categoria|latitud|longitud|propietario|explotador|longitud_pista|ancho_pista|pbmo|elevacion|resolucion|fecha_construccion|fecha_vigencia|clase|tipo|numero_vuelos_origen|gcd_departamento|gcd_municipio|
+---+-----+----+------+---------+------------+---------+-------+--------+-----------+----------+--------------+-----------+----+---------+----------+------------------+--------------+-----+----+--------------------+----------------+-------------+
|  0|    0| 209|     0|        0|           0|        0|      0|       0|          3|         0|             0|          0|  55|        0|         6|                 0|           221|    0|   0|                  71|               0|            0|
+---+-----+-

In [9]:
aeropuertos = aeropuertos.withColumn("pbmo",aeropuertos["pbmo"].cast(IntegerType()))
aeropuertos = aeropuertos.withColumn("numero_vuelos_origen",aeropuertos["numero_vuelos_origen"].cast(IntegerType()))
aeropuertos = aeropuertos.withColumn("fecha_construccion",aeropuertos["fecha_construccion"].cast(DateType()))
aeropuertos = aeropuertos.withColumn("fecha_vigencia",aeropuertos["fecha_vigencia"].cast(DateType()))
aeropuertos = aeropuertos.withColumn("clase",aeropuertos["clase"].cast(IntegerType()))

In [10]:
aeropuertos.select('latitud', 'longitud', 'longitud_pista', 'ancho_pista', 'pbmo', 'elevacion', 'resolucion', 'clase', 'numero_vuelos_origen', 'gcd_departamento', 'gcd_municipio').summary().show()

+-------+------------------+------------------+-----------------+------------------+-----------------+------------------+----------+-----+--------------------+-----------------+------------------+
|summary|           latitud|          longitud|   longitud_pista|       ancho_pista|             pbmo|         elevacion|resolucion|clase|numero_vuelos_origen| gcd_departamento|     gcd_municipio|
+-------+------------------+------------------+-----------------+------------------+-----------------+------------------+----------+-----+--------------------+-----------------+------------------+
|  count|               292|               292|              292|               292|              237|               292|       292|    0|                 221|              292|               292|
|   mean| 5.543335616438355|-73.24611404109585| 962.445205479452|17.633561643835616|5683.341772151899|1073.3082191780823|       NaN| null|  185576.36199095022|63.03767123287671| 63341.63698630137|
| stddev|2.3826

In [11]:
print("sigla: ", aeropuertos.select("sigla").distinct().count())
print("iata: ", aeropuertos.select("iata").distinct().count())
print("nombre: ", aeropuertos.select("nombre").distinct().count())
print("municipio: ", aeropuertos.select("municipio").distinct().count())
print("departamento: ", aeropuertos.select("departamento").distinct().count())
print("categoria: ", aeropuertos.select("categoria").distinct().count())
print("propietario: ", aeropuertos.select("propietario").distinct().count())
print("explotador: ", aeropuertos.select("explotador").distinct().count())
print("tipo: ", aeropuertos.select("tipo").distinct().count())

sigla:  212
iata:  62
nombre:  206
municipio:  122
departamento:  30
categoria:  4
propietario:  121
explotador:  103
tipo:  4


In [13]:
aeropuertos = aeropuertos.fillna({'numero_vuelos_origen': 185576})

In [14]:
vuelos = spark.read.load(PATH+"vuelos.csv",format="csv", sep=",", inferSchema="true", header="true")
vuelos.show()

+----+---+------+-------+-----------+----------+-------+--------------------+------+------+--------------+---------+------------------+
| ano|mes|origen|destino|tipo_equipo|tipo_vuelo|trafico|             empresa|vuelos|sillas|carga_ofrecida|pasajeros|       carga_bordo|
+----+---+------+-------+-----------+----------+-------+--------------------+------+------+--------------+---------+------------------+
|2012|  1|   BOG|    CUC|       BE20|         T|      N|             RIO SUR|   1.0|   0.0|           0.0|      4.0|             100.0|
|2013|  5|   UIB|    BOG|       DH8D|         R|      N|               AIRES|  30.0|1110.0|       24000.0|    873.0|            4222.0|
|2013| 10|   IBE|    BOG|       DH8D|         R|      N|               AIRES|  98.0|3626.0|       56056.0|   2866.0|           2323.75|
|2012|  4|   FLA|    BOG|       JS32|         T|      N|         SARPA S.A.S|   1.0|   0.0|           0.0|      4.0|               0.0|
|2013|  7|   CUC|    AUC|       C402|         T|

In [15]:
print(vuelos.schema)

StructType(List(StructField(ano,IntegerType,true),StructField(mes,IntegerType,true),StructField(origen,StringType,true),StructField(destino,StringType,true),StructField(tipo_equipo,StringType,true),StructField(tipo_vuelo,StringType,true),StructField(trafico,StringType,true),StructField(empresa,StringType,true),StructField(vuelos,StringType,true),StructField(sillas,StringType,true),StructField(carga_ofrecida,DoubleType,true),StructField(pasajeros,StringType,true),StructField(carga_bordo,DoubleType,true)))


In [16]:
row_number = vuelos.count()
print(row_number)

67599


In [17]:
vuelos.select([count(when(isnan(c), c)).alias(c) for c in vuelos.columns]).show()

+---+---+------+-------+-----------+----------+-------+-------+------+------+--------------+---------+-----------+
|ano|mes|origen|destino|tipo_equipo|tipo_vuelo|trafico|empresa|vuelos|sillas|carga_ofrecida|pasajeros|carga_bordo|
+---+---+------+-------+-----------+----------+-------+-------+------+------+--------------+---------+-----------+
|  0|  0|     0|      0|          0|         0|    894|      0|  1582|  4017|             0|     3483|          0|
+---+---+------+-------+-----------+----------+-------+-------+------+------+--------------+---------+-----------+



In [18]:
vuelos = vuelos.withColumn("tipo_equipo",vuelos["tipo_equipo"].cast(IntegerType()))
vuelos = vuelos.withColumn("vuelos",vuelos["vuelos"].cast(IntegerType()))
vuelos = vuelos.withColumn("sillas",vuelos["sillas"].cast(IntegerType()))
vuelos = vuelos.withColumn("pasajeros",vuelos["pasajeros"].cast(IntegerType()))

In [19]:
vuelos.select('vuelos', 'sillas', 'carga_ofrecida','pasajeros', 'carga_bordo',).summary().show()

+-------+------------------+------------------+------------------+-----------------+------------------+
|summary|            vuelos|            sillas|    carga_ofrecida|        pasajeros|       carga_bordo|
+-------+------------------+------------------+------------------+-----------------+------------------+
|  count|             66017|             63582|             67599|            64116|             67599|
|   mean|14.263735098535225|1133.1225346796264| 46758.48664891492|911.7284297211304| 5325.677899968939|
| stddev| 40.19851542146546| 4468.301901592545|230667.33670267835| 3613.76621059051|23415.901919555745|
|    min|                 1|                 0|               0.0|                0|               0.0|
|    25%|                 1|                 0|               0.0|                2|               0.0|
|    50%|                 2|                 0|               0.0|                8|             110.0|
|    75%|                 9|               240|            9000.

In [20]:
print("origen: ", vuelos.select("origen").distinct().count())
print("destino: ", vuelos.select("destino").distinct().count())
print("tipo_equipo: ", vuelos.select("tipo_equipo").distinct().count())
print("tipo_vuelo: ", vuelos.select("tipo_vuelo").distinct().count())
print("trafico: ", vuelos.select("trafico").distinct().count())
print("empresa: ", vuelos.select("empresa").distinct().count())

origen:  111
destino:  110
tipo_equipo:  7
tipo_vuelo:  4
trafico:  2
empresa:  71
